In [7]:
import json
import numpy as np
from scipy.spatial.transform import Rotation as R

with open("../drawing/drawing_config.json", "r") as f:
    config = json.load(f)

T, u, v, normal, origin = np.array(config["T"]), np.array(config["u"]), np.array(config["v"]), np.array(config["normal"]), np.array(config["origin"])

poses = config["poses"]
print("origin: ", origin)
print("u: ", u)
print("v: ", v)
print("normal: ", normal)
print("")


origin:  [ 0.23817554  0.01953333 -0.02409203]
u:  [ 0.35047046 -0.93515956  0.05144949]
v:  [0.93474657 0.35268831 0.04312556]
normal:  [-0.05847492  0.032978    0.99774402]



In [8]:

def plot_poses(T, u,v,origin, normal, poses):
    import plotly.graph_objects as go
    from scipy.spatial.transform import Rotation as R

    def arrow(start, vec, color, name=""):
        return go.Scatter3d(
            x=[start[0], start[0] + vec[0]],
            y=[start[1], start[1] + vec[1]],
            z=[start[2], start[2] + vec[2]],
            mode="lines+markers",
            line=dict(color=color, width=4),
            marker=dict(size=3, color=color),
            name=name
        )

    fig = go.Figure()

    # Axes
    fig.add_trace(arrow(origin, u * 0.01, "red", "u"))
    fig.add_trace(arrow(origin, v * 0.01, "green", "v"))
    fig.add_trace(arrow(origin, normal * 0.01, "black", "normal"))

    # pose positions
    for i, s in enumerate(poses):
        pos = np.array([s["ee.x"], s["ee.y"], s["ee.z"]])
        rotvec = np.array([s["ee.wx"], s["ee.wy"], s["ee.wz"]])

        # --- Convert to rotation matrix ---
        rot = R.from_rotvec(rotvec)
        R_mat = rot.as_matrix()

        # --- Original coordinate axes ---
        x_axis = np.array([1, 0, 0])
        y_axis = np.array([0, 1, 0])
        z_axis = np.array([0, 0, 1])

        # --- Rotated axes ---
        x_rot = R_mat @ x_axis
        y_rot = R_mat @ y_axis
        z_rot = R_mat @ z_axis
        fig.add_trace(arrow(pos, x_rot * 0.01, 'red'))
        fig.add_trace(arrow(pos, y_rot * 0.01, 'blue'))
        fig.add_trace(arrow(pos, z_rot * 0.01, 'green'))

    # Layout
    r = 0.5
    fig.update_layout(
        scene=dict(
            xaxis=dict(range=[origin[0] - r, origin[0] + r], title='X'),
            yaxis=dict(range=[origin[1] - r, origin[1] + r], title='Y'),
            zaxis=dict(range=[origin[2] - r, origin[2] + r], title='Z'),
            aspectmode='cube'
        ),
        margin=dict(l=0, r=0, t=40, b=0),
        title="Interactive TCP Visualization",
    )
    fig.show()

plot_poses(T, u, v, origin, normal, poses)
print("num poses: ", len(poses))

num poses:  8


In [9]:
import math


def nearest_equivalent_rotvec(rotvec_source, rotvec_ref):
    """
    Find an equivalent rotvec to rotvec_source (same rotation) that is
    closest in Euclidean distance to rotvec_ref by trying +/- 2π * axis.
    """
    # source axis & angle
    angle = np.linalg.norm(rotvec_source)
    if angle < 1e-12:
        return rotvec_source.copy()
    axis = rotvec_source / angle

    # try offsets k*2π where k in {-1,0,1}
    best = None
    best_d = None
    for k in [-1, 0, 1]:
        cand = axis * (angle + 2 * math.pi * k)
        d = np.linalg.norm(cand - rotvec_ref)
        if best is None or d < best_d:
            best = cand
            best_d = d
    return best


# example: sampled rotvec and generated rotvec (from your message)
rotvec_sample = np.array([-0.5778150380753542, 2.620369318561434, 0.141891978245403])
# this is what your code produced
rotvec_generated = np.array([-2.576451713041018, 1.7837453732352375, -0.10485935865368333])

# usage:
rotvec_g_equiv = nearest_equivalent_rotvec(rotvec_generated, rotvec_sample)
print("Generated rotvec (original):", rotvec_generated)
print("Generated rotvec (equiv nearest):", rotvec_g_equiv)
print("Distance to sample before/after:", np.linalg.norm(rotvec_generated - rotvec_sample),
                                      np.linalg.norm(rotvec_g_equiv - rotvec_sample))

Generated rotvec (original): [-2.57645171  1.78374537 -0.10485936]
Generated rotvec (equiv nearest): [-2.57645171  1.78374537 -0.10485936]
Distance to sample before/after: 2.1806820966551865 2.1806820966551865


In [10]:
plane_u = u / np.linalg.norm(u)
plane_v = v / np.linalg.norm(v)
plane_normal = normal / np.linalg.norm(normal)

R_base = np.column_stack([plane_u, plane_v, plane_normal])
r_v = R.from_matrix(R_base).as_rotvec()

rotvec_sample = np.array([-0.5778150380753542, 2.620369318561434, 0.141891978245403])

rotvec_g_equiv = nearest_equivalent_rotvec(rotvec_sample, r_v)
print("Generated rotvec (equiv nearest):", r_v)
print("Distance ", np.linalg.norm(rotvec_sample - r_v))

Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095


In [11]:
def rotvec_to_mat(rv):
    return R.from_rotvec(rv).as_matrix()

def rotation_difference_angle(rv_a, rv_b):
    """Physical rotation angle (deg) between two rotvecs: angle of R_b * R_a^T."""
    Ra = rotvec_to_mat(rv_a)
    Rb = rotvec_to_mat(rv_b)
    R_rel = Rb @ Ra.T
    # clamp trace
    tr = np.trace(R_rel)
    cos_theta = max(min((tr - 1.0) / 2.0, 1.0), -1.0)
    theta = math.acos(cos_theta)
    return theta * 180.0 / math.pi, R_rel

diff = rotation_difference_angle(r_v, rotvec_sample)
print("Rotation difference (deg):", diff[0])

Rotation difference (deg): 165.81719302271802


In [12]:
from drawing.utils import compose_ee_pose

# create 2D circle path in meters, center (0,0) radius 0.05
angles = np.linspace(0, 2 * np.pi, 300)
circle2d = [(0.05 * np.cos(a), 0.05 * np.sin(a)) for a in angles]
hover_h = 0.1

for s, t in circle2d:
    pos_w = origin + s * u + t * v + hover_h * -normal

    R_base = np.column_stack([plane_u, plane_v, plane_normal])
    r_v = R.from_matrix(R_base).as_rotvec()

    # action = compose_ee_pose(pos_w, u, v, normal)
    # r_v = np.array([action["ee.wx"], action["ee.wy"], action["ee.wz"]])
    rotvec_g_equiv = nearest_equivalent_rotvec(rotvec_sample, r_v)
    print("Generated rotvec (equiv nearest):", r_v)
    print("Distance ", np.linalg.norm(rotvec_sample - r_v))

Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217

In [15]:
def plane_aligned_rotvec(points, ref_rotvec=None):
    """
    Compute a rotation vector for a robot EE aligned to a plane defined by points.

    Parameters
    ----------
    points : (N,3) array
        Sampled points on a plane.
    ref_rotvec : (3,) array, optional
        Reference rotation vector to canonicalize the output for smoothness.

    Returns
    -------
    rot_vec : (3,) array
        Rotation vector [wx, wy, wz] aligned to the plane (z along -normal).
    """
    points = np.asarray(points)
    if points.shape[0] < 3:
        raise ValueError("Need at least 3 points to define a plane")

    # --- Step 1: Plane fitting (SVD) ---
    centroid = points.mean(axis=0)
    Q = points - centroid
    _, _, vh = np.linalg.svd(Q, full_matrices=False)
    normal = vh[-1, :]
    normal /= np.linalg.norm(normal)

    # --- Step 2: Define in-plane axes ---
    tmp = points[1] - points[0]
    u = tmp - np.dot(tmp, normal) * normal
    u /= np.linalg.norm(u)
    v = np.cross(normal, u)

    # --- Step 3: Build right-handed EE frame ---
    z_axis = -normal          # EE z points down the plane normal
    y_axis = np.cross(z_axis, u)
    y_axis /= np.linalg.norm(y_axis)
    x_axis = np.cross(y_axis, z_axis)
    x_axis /= np.linalg.norm(x_axis)

    R_base = np.column_stack([x_axis, y_axis, z_axis])
    if np.linalg.det(R_base) < 0:
        R_base[:, 2] *= -1  # ensure right-handed frame

    # --- Step 4: Convert to rotation vector ---
    rot_vec = R.from_matrix(R_base).as_rotvec()

    # --- Step 5: Optional canonicalization ---
    if ref_rotvec is not None:
        rot_vec = nearest_equivalent_rotvec(rot_vec, ref_rotvec)

    return rot_vec

r_v = plane_aligned_rotvec([origin, origin + u, origin + v])
rotvec_g_equiv = nearest_equivalent_rotvec(rotvec_sample, r_v)
print("Generated rotvec (equiv nearest):", r_v)

print("Distance ", np.linalg.norm(rotvec_sample - r_v))

# your plane rotation vector
rv_plane = r_v  # from plane_aligned_rotvec

# convert to rotation matrix
R_plane = R.from_rotvec(rv_plane).as_matrix()
# flip the EE z-axis
R_flipped = R_plane.copy()
R_flipped[:, 2] *= -1  # invert the z-axis
# re-orthonormalize x and y to keep a proper right-handed frame
R_flipped[:, 0] = np.cross(R_flipped[:, 1], R_flipped[:, 2])
R_flipped[:, 0] /= np.linalg.norm(R_flipped[:, 0])
R_flipped[:, 1] = np.cross(R_flipped[:, 2], R_flipped[:, 0])
R_flipped[:, 1] /= np.linalg.norm(R_flipped[:, 1])

# convert back to rotation vector
rv_flipped = R.from_matrix(R_flipped).as_rotvec()


Generated rotvec (equiv nearest): [ 0.00656984 -0.07116849 -1.21063546]
Distance  3.0684217297384095
Generated rotvec (equiv nearest): [1.75016973 2.52795487 0.00949777]
Flipped Distance  2.3335770245467837
